# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [6]:
# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

In [3]:
spark = SparkSession \
    .builder \
    .appName("Wrangling DataFrames") \
    .getOrCreate()

df = spark.read.json("data/sparkify_log_small.json")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/29 13:35:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/29 13:35:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/08/29 13:35:27 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/08/29 13:35:27 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
                                                                                

In [4]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id "" (empty string) NOT visit?

In [8]:
# TODO: write your code to answer question 1
blank_pages = df.filter(df.userId == '') \
    .select(col('page').alias('blankPages')) \
    .dropDuplicates()

all_pages = df.select('page').dropDuplicates()

for row in set(all_pages.collect()) - set(blank_pages.collect()):
    print(row.page)

Settings
Upgrade
Error
Logout
Submit Downgrade
Downgrade
Submit Upgrade
Save Settings
NextSong


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [2]:
# TODO: use this space to explore the behavior of the user with an empty string


# Question 3

How many female users do we have in the data set?

In [3]:
# TODO: write your code to answer question 3

# Question 4

How many songs were played from the most played artist?

In [ ]:
# TODO: write your code to answer question 4

# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [ ]:
# TODO: write your code to answer question 5